# Data Cleaning
The cleaning process is organized into sections, each corresponding to a different dataset (`.csv` file). Each section includes the following steps:

1. **Data Understanding**: Initial exploration of the dataset.
2. **Data Cleaning**: Handling of missing values (NaN), removal of duplicates, setting correct data types, and renaming columns.
   *(Optional)* **Deep Clean**: Custom cleaning steps applied to a specific dataset, if necessary.
3. **Final Result**: Displays the cleaned dataset and saves it to a new `.csv` file.

First, import the necessary libraries and set up any required options.

In [ ]:
import pandas as pd
import numpy as np

from utils.utils import find_matching

# Set to True to print cleaned data into new csv
PRINT_CSV = False

## Movies

In [ ]:
# Import 'movies.csv' dataset
movies_df = pd.read_csv('datasets/movies.csv')

### 1. Data Understanding

In [ ]:
movies_df.head()

In [ ]:
movies_df.shape

In [ ]:
movies_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
movies_df = movies_df.rename(columns={'name': 'title', 'minute': 'duration_in_minutes', 'date': 'release_year'})
print(f"Movies dataset columns: {', '.join(movies_df.columns)}")

In [ ]:
# Check for null values
movies_df.isna().sum()

There are null values in most of the columns.
The fields '**release_year**', '**tagline**', '**description**', '**duration_in_minutes**' and '**rating**' don't cause any problems so we'll keep them, but the few movies that are without a title can't be used and will be removed.

In [ ]:
# Removing rows with null title
no_title = movies_df[movies_df['title'].isna()]
movies_df = movies_df.dropna(subset=['title'])

print("Movies dataset without title:")
no_title

In [ ]:
# Check for duplicate rows
if movies_df.duplicated().sum():
    print(f"There are {movies_df.duplicated().sum()} duplicated rows")
else:
    print("No duplicated rows")

In [ ]:
# Check if 'id' column has unique values
duplicates_id = movies_df[movies_df['id'].duplicated()].shape[0]
print(f"'id' duplicates: {duplicates_id}")

The '**id**' field is the dataset's Primary Key, so we'll set it as the index.

In [ ]:
movies_df = movies_df.set_index("id")

In [ ]:
# Setting the correct type for columns
movies_df['release_year'] = movies_df['release_year'].astype('Int64')
movies_df['duration_in_minutes'] = movies_df['duration_in_minutes'].astype('Int64')
movies_df[['release_year', 'duration_in_minutes']].dtypes

#### Deep Clean
Let's look inside some columns to see most frequent values

In [ ]:
movies_df['description'].value_counts().head(10)

Many descriptions seem to have a description like "Plot Unavailable" or similar instead of a null value. The other fields seem fine.<br>
Let's try to fix as many as possible (fixing only the most frequent variation, not 100% accurate).

In [ ]:
from utils.utils import null_movie_description_keywords

# Find null description variation
result = find_matching(movies_df, 'description', null_movie_description_keywords, max_length=30)
matches = result.copy()

# Fill with NaN values the result obtained
result['description'] = np.nan

# Manual check to be sure to not overwrite real descriptions
matches['description'].value_counts()

### 3. Final Result
All datasets reference the **movies** dataset. A movie is uniquely identified by his **id** and a movie id has multiple occurrences in other datasets.

In [ ]:
movies_df.head()

In [ ]:
movies_df.shape

In [ ]:
# Print clean dataset to new csv file
if PRINT_CSV:
    movies_df.to_csv('clean_datasets/movies.csv')

# Free memory
movies_df = None

## Languages

In [ ]:
# Import 'languages.csv' dataset
lang_df = pd.read_csv('datasets/languages.csv')

### 1. Data Understanding

In [ ]:
lang_df.head()

In [ ]:
lang_df.shape

In [ ]:
lang_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
lang_df = lang_df.rename(columns={'id': 'movie_id'})
print(f"Languages dataset columns: {', '.join(lang_df.columns)}")

In [ ]:
# Check for null values
lang_df.isna().sum()

In [ ]:
# Check for duplicate rows
lang_df.duplicated().sum()

In [ ]:
# Setting the category data type for column 'type'
lang_types = lang_df['type'].unique()
lang_df['type'] = lang_df['type'].astype('category')
print(f"types: {', '.join(lang_types)}")

The '**type**' field has only 3 possible values, so we can set it as a categorical type

### 3. Final Result
The languages dataset is directly connected to the movies dataset with the movie_id column. There are more languages rows than movies rows, because a movie can have multiple languages connected. Also, not all movie must have a language defined.

In [ ]:
lang_df.head()

In [ ]:
lang_df.shape

In [ ]:
# Print clean dataset to new csv file
if PRINT_CSV:
    lang_df.to_csv('clean_datasets/languages.csv')

# Free memory
lang_df = None

## Actors

In [ ]:
# Import the 'actors.csv' dataset
actors_df = pd.read_csv('datasets/actors.csv')

### 1. Data Understanding

In [ ]:
actors_df.head()

In [ ]:
actors_df.shape

In [ ]:
actors_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
actors_df = actors_df.rename(columns={'id': 'movie_id'})

In [ ]:
# Check for null values
actors_df.isna().sum()

There are a lot of missing roles, but there is nothing to do about it.

In [ ]:
# Few actors are without a name and can't be used. Remove them
no_name = actors_df[actors_df['name'].isna()]
actors_df = actors_df.dropna(subset=['name'])
no_name

In [ ]:
# Check for duplicate rows
print('Duplicated rows:', actors_df.duplicated().sum())
actors_duplicates = actors_df[actors_df.duplicated(keep=False)].head(6)

# Dropping the duplicates
actors_df = actors_df.drop_duplicates()

actors_duplicates

#### Deep Clean

In [ ]:
actors_df['role'].value_counts().head(10)

The role column has many "Self" role variations let's look more deeply.

In [ ]:
from utils.utils import self_actor_role_keywords

# Find self variation
result = find_matching(actors_df, 'role', self_actor_role_keywords)
print('Values matching:', result['role'].shape[0])
result['role'].value_counts().head(10)

There are over 300.000 values that are similar to "Self", but many of them contains also other information as "Self - Presenter" or "Self - Guest". Overwriting all those values could result in a loss of information, so they won't be overwritten in the cleaned dataset, but they might be when visualizing the data for statistical purposes.

In [ ]:
# Reset indexing after removing rows
actors_df = actors_df.reset_index(drop=True)

### 3. Final Result
The actors dataset is directly connected to the movies dataset and has almost six times the number of rows as the movies dataset. Also, a movie can have no actors connected

In [ ]:
actors_df.head()

In [ ]:
actors_df.shape

In [ ]:
# Print clean dataset to new csv file
if PRINT_CSV:
    actors_df.to_csv('clean_datasets/actors.csv')

# Free memory
actors_df = None

## Countries

In [ ]:
# Import 'countries.csv' dataset
countries_df = pd.read_csv('datasets/countries.csv')

### 1. Data Understanding

In [ ]:
countries_df.head()

In [ ]:
countries_df.shape

In [ ]:
countries_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
countries_df = countries_df.rename(columns={'id': 'movie_id'})
print(f"Countries dataset columns: {', '.join(countries_df.columns)}")

In [ ]:
# Check for null values
countries_df.isna().sum()

In [ ]:
# Check for duplicate rows
countries_df.duplicated().sum()

### 3. Final Results
The **countries** dataset is directly connected to the movies dataset with the movie_id column as a Foreign Key. This dataset contains all the countries where the movies were produced.


In [ ]:
countries_df.head()

In [ ]:
countries_df.shape

In [ ]:
# Print clean dataset to new csv file
if PRINT_CSV:
    countries_df.to_csv('clean_datasets/countries.csv')

# Free memory
countries_df = None

## Crew

In [ ]:
# Import 'crew.csv' dataset
crew_df = pd.read_csv('datasets/crew.csv')

### 1. Data Understanding

In [ ]:
crew_df.head()

In [ ]:
crew_df.shape

In [ ]:
crew_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
crew_df = crew_df.rename(columns={'id': 'movie_id', 'name': 'crew_member_name'})
print(f"Crew dataset columns: {', '.join(crew_df.columns)}")

In [ ]:
# Check for null values
crew_df.isna().sum()

The existing `NaN` value was not removed because it is minimal compared to the overall size of the dataset, and removing it would not significantly impact the analysis. Similarly, values such as *Unknown* or *Anonymous* were kept because they account for less than 1% of the data and do not affect the overall results.

In [ ]:
# Check for duplicate rows
print('Duplicated rows:', crew_df.duplicated().sum())
crew_duplicates = crew_df[crew_df.duplicated(keep=False)].head()

# Dropping the duplicates
crew_df = crew_df.drop_duplicates()

crew_duplicates

#### Deep Clean

In [ ]:
# TODO:

### 3. Final Results
The **crew** dataset is connected to the movies dataset through the 'movie_id' Foreign Key. It includes the names of all crew members along with their roles. A crew member can have multiple roles, but cannot perform the same role in the same movie more than once.

In [ ]:
crew_df.head()

In [ ]:
crew_df.shape

In [ ]:
# Print clean dataset to new csv file
if PRINT_CSV:
    crew_df.to_csv('clean_datasets/crew.csv')

# Free memory
crew_df = None

## Genres

In [ ]:
# Import 'genres.csv' dataset
genres_df = pd.read_csv('datasets/genres.csv')

### 1. Data Understanding

In [ ]:
genres_df.head()

In [ ]:
genres_df.shape

In [ ]:
genres_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
genres_df = genres_df.rename(columns={'id': 'movie_id'})
print(f"Genres dataset columns: {', '.join(genres_df.columns)}")

In [ ]:
# Check for null values
genres_df.isna().sum()

In [ ]:
# Check for duplicate rows
genres_df.duplicated().sum()

In [ ]:
# Setting the correct type for columns
genres_list = list(genres_df["genre"].unique())
print(f'There are {len(genres_list)} genres in the dataset: {", ".join(genres_list)}')

genres_df['genre'] = genres_df['genre'].astype('category')

### 3. Final Results
The **genres** dataset is connected to the movies dataset through the movie_id Foreign Key. A movie can have multiple genres.


In [ ]:
genres_df.head()

In [ ]:
genres_df.shape

In [ ]:
# Print clean dataset to new csv file
if PRINT_CSV:
    genres_df.to_csv('clean_datasets/genres.csv')

# Free memory
genres_df = None

## Posters

In [ ]:
# Import 'posters.csv' dataset
posters_df = pd.read_csv('datasets/posters.csv')

### 1. Data Understanding

In [ ]:
posters_df.head()

In [ ]:
posters_df.shape

In [ ]:
posters_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
posters_df = posters_df.rename(columns={'id': 'movie_id', 'link': 'poster_link'})
print(f"Posters dataset columns: {', '.join(posters_df.columns)}")

In [ ]:
# Check for null values
posters_df.isna().sum()

In [ ]:
# Removing null rows
posters_df = posters_df.dropna()

It was decided to remove the `NaN` values as they do not contribute meaningful information to the dataset and could hinder data consistency and analysis.

In [ ]:
# Check for duplicate rows
posters_df.duplicated().sum()

In [ ]:
# Check if a movie can have more than 1 poster
id_duplicates = posters_df['movie_id'].duplicated().any()

if id_duplicates:
    print("There are duplicates in the 'movie_id' column.")
else:
    print("There are no duplicates in the 'movie_id' column.")

The relationship between the **posters** dataset and the **movies** dataset must be One-to-One, allowing us to consider merging the two datasets.

In [ ]:
from utils.utils import check_valid_links

print(check_valid_links(posters_df, 'poster_link'))

### 3. Final Results

In [ ]:
posters_df.head()

In [ ]:
posters_df.shape

In [ ]:
# TODO: Merging the datasets on 'id' from 'movies' and 'movie_id' from 'posters'
# merged_df = pd.merge(movies_df, posters_df, left_on='id', right_on='movie_id', how='left')

In [ ]:
if PRINT_CSV:
    posters_df.to_csv('clean_datasets/posters.csv')

posters_df = None

## Releases

In [ ]:
# Import the 'release.csv' dataset
releases_df = pd.read_csv('datasets/releases.csv')

### 1. Data Understanding

In [ ]:
releases_df.head()

In [ ]:
releases_df.shape

In [ ]:
releases_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
releases_df = releases_df.rename(columns={'id': 'movie_id', 'type': 'distribution_format'})
print(f"Release dataset columns: {', '.join(releases_df.columns)}")

In [ ]:
# Check for null values
releases_df.isna().sum()

In [ ]:
# Check for duplicate rows
releases_df.duplicated().sum()

In [ ]:
releases_df[(releases_df['rating'] == "0") & (~releases_df['country'].isin(["Germany", "Austria", "Switzerland"]))]

We initially checked the value `0` in the dataset, assuming it might represent a null or missing value. However, we discovered that in Germany, Austria, and Switzerland, the `0` rating has a meaningful interpretation, indicating that the film is suitable for all audiences, including children. For the remaining countries, it is possible that the `0` rating is an error, but since it appears only 70 times in a dataset of 1,332,782 rows, we deemed it unnecessary to remove or correct these entries.

In [ ]:
# Setting the correct type for the columns
releases_df['date'] = pd.to_datetime(releases_df['date'], format='%Y-%m-%d')
releases_df['distribution_format'] = releases_df['distribution_format'].astype('category')

### 3. Final Results
The **release** dataset is linked to the movies dataset via the Foreign Key movie_id. It contains details about the movies' releases around the world.

In [ ]:
releases_df.head()

In [ ]:
releases_df.shape

In [ ]:
if PRINT_CSV:
    releases_df.to_csv('clean_datasets/releases.csv')

releases_df = None

## Studios

In [ ]:
# Import the 'studios.csv' dataset
studios_df = pd.read_csv('datasets/studios.csv')

### 1. Data Understanding

In [ ]:
studios_df.head()

In [ ]:
studios_df.shape

In [ ]:
studios_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
studios_df = studios_df.rename(columns={'id': 'movie_id'})

In [ ]:
# Check for null values
studios_df.isna().sum()

# Dropping rows with null studios
studios_df = studios_df.dropna()

In [ ]:
# Check for duplicate row
studios_df.duplicated().sum()

# Dropping duplicate rows
studios_df = studios_df.drop_duplicates()

### 3. Final Results
The **studios** dataset is linked to the movies dataset via the Foreign Key movie_id.

In [ ]:
studios_df.head()

In [ ]:
studios_df.shape

In [ ]:
if PRINT_CSV:
    studios_df.to_csv('clean_datasets/studios.csv')

studios_df = None

## Themes

In [ ]:
# Import the 'themes.csv' dataset
themes_df = pd.read_csv('datasets/themes.csv')

### 1. Data Understanding

In [ ]:
themes_df.head()

In [ ]:
themes_df.shape

In [ ]:
themes_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
themes_df = themes_df.rename(columns={'id': 'movie_id'})

In [ ]:
# Check for null values
themes_df.isna().sum()

In [ ]:
# Check for duplicate rows
themes_df.duplicated().sum()

In [ ]:
# Check for invalid themes
themes_df['theme'].value_counts()

### 3. Final Results
The **themes** dataset is linked to the movies dataset via the Foreign Key movie_id. It describes a movie with few, standard phrases.

In [ ]:
themes_df.head()

In [ ]:
themes_df.shape

## The Oscar Awards

In [ ]:
# Import the 'the_oscar_awards.csv' dataset
oscars_df = pd.read_csv('datasets/the_oscar_awards.csv')

### 1. Data Understanding

In [ ]:
oscars_df.head()

In [ ]:
oscars_df.shape

In [ ]:
oscars_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
oscars_df = oscars_df.rename(columns={'name': 'nominee_name', 'film': 'nominated_film', 'winner': 'is_winner'})

In [ ]:
# Check for null values
oscars_df.isna().sum()

In [ ]:
# Check for duplicate rows
oscars_df.duplicated().sum()

In [ ]:
# Check the consistency between year_film and year_ceremony
oscars_df[oscars_df['year_film'] > oscars_df['year_ceremony']]

In [ ]:
# Check for multiple winner possibilities
from utils.utils import special_oscar_awards

# Exclude rows where the 'category' is in the special_oscar_awards list
filtered_oscars = oscars_df[~oscars_df['category'].isin(special_oscar_awards)]

# Find groups with more than one winner
multiple_winners = filtered_oscars.groupby(['year_ceremony', 'category']).filter(
    lambda x: x['is_winner'].sum() > 1
)

# Keep only the rows where 'is_winner' is True
multiple_winners = multiple_winners[multiple_winners['is_winner'] == True]
multiple_winners.head()

Displaying multiple winners (including draws), as we want to keep all winners in case of a draw in the same category (1969 Best Actress, or 1932 Best Actor)

In [ ]:
duplicates_nominee = oscars_df[oscars_df.duplicated(subset=['nominee_name', 'nominated_film', 'category', 'ceremony'], keep=False)]
duplicates_nominee.head()

Duplicates are retained because, in music categories, the same artists can receive identical nominations for different songs, with the song titles not specified in the dataset

In [ ]:
# Setting the correct type for columns
oscars_df['category'] = oscars_df['category'].astype('category')

### 3. Final Results

In [ ]:
oscars_df.head()

In [ ]:
oscars_df.shape

In [ ]:
if PRINT_CSV:
    oscars_df.to_csv('clean_datasets/the_oscar_awards.csv')

oscars_df = None

## Rotten Tomatoes Reviews

In [ ]:
reviews_df = pd.read_csv('datasets/rotten_tomatoes_reviews.csv')

### 1. Data Understanding

In [ ]:
reviews_df.head()

In [ ]:
reviews_df.shape

In [ ]:
reviews_df.dtypes

### 2. Data Cleaning

In [ ]:
# Rename columns
reviews_df = reviews_df.rename(columns={'review_type': 'type', 'review_score': 'score', 'review_date': 'date', 'review_content': 'content', 'top_critic': 'is_top_critic'})

In [ ]:
# Check for null values
reviews_df.isna().sum()

Checking on Rotten Tomatoes website is fine having the publisher and critic name and content as null values.

In [ ]:
# Check for duplicate rows
filtered_df = reviews_df[reviews_df['critic_name'].notna()]
filtered_df = filtered_df[filtered_df.duplicated(keep=False)]

reviews_df = reviews_df.drop(reviews_df[reviews_df['critic_name'].isna()].index)

filtered_df

There are many duplicate reviews in the datasets. Looking closely is actually possible to have more reviews for the same movie that have the same publisher and with the author not specified. Those rows will be excluded from the total count of duplicate rows and will not be removed. All other duplicate rows will be removed.

In [ ]:
# Setting the correct type for columns
reviews_df['type'] = reviews_df['type'].astype('category')
reviews_df['date'] = pd.to_datetime(reviews_df['date'], format='%Y-%m-%d')

### 3. Final Result

In [ ]:
reviews_df.head()

In [ ]:
reviews_df.shape

In [ ]:
# Print clean dataset to new csv file
if PRINT_CSV:
    reviews_df.to_csv('clean_datasets/reviews.csv')

# Free memory
reviews_df = None